## Note
In this notebook we will load a trained GMF++ model, and go over the evaluation procedure. The GMF++ is based on simple model introduced by [He et al](https://arxiv.org/abs/1708.05031). You can try to adapt other models such as MLP and NMF. The [original implementation](https://github.com/hexiangnan/neural_collaborative_filtering/tree/4aab159e81c44b062c091bdaed0ab54ac632371f) as well as other implemntations are available for single market settings.     

In [1]:
import argparse
import pandas as pd
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, ConcatDataset

import os
from os import path
import json
import resource
import sys
import pickle
from zipfile import ZipFile

sys.path.insert(1, 'src')
from model import Model
from utils import *
from data import *
from train_baseline import *

In [8]:
parser = create_arg_parser()

tgt_market = 't1' 
src_markets = 'none' # 'none' | 's1' | 's1_s2_s3'
exp_names = 'toytest'
tgt_market_valid = f'DATA/{tgt_market}/valid_run.tsv'
tgt_market_test = f'DATA/{tgt_market}/test_run.tsv'


args = parser.parse_args(f'--tgt_market {tgt_market} --src_markets {src_markets} \
            --tgt_market_valid {tgt_market_valid} --tgt_market_test {tgt_market_test} --cuda'.split()) #

if torch.cuda.is_available() and args.cuda:
    torch.cuda.set_device(0)
args.device = torch.device('cuda' if torch.cuda.is_available() and args.cuda else 'cpu')
print("Device:", args.device)

Device: cpu


In [9]:
# load pretrained model
model_dir = f'checkpoints/{tgt_market}_{src_markets}_{exp_names}.model'
id_bank_dir = f'checkpoints/{tgt_market}_{src_markets}_{exp_names}.pickle'

with open(id_bank_dir, 'rb') as centralid_file:
    my_id_bank = pickle.load(centralid_file)

mymodel = Model(args, my_id_bank)
mymodel.load(model_dir)

Model is GMF++!
GMF(
  (embedding_user): Embedding(2698, 8)
  (embedding_item): Embedding(1386, 8)
  (affine_output): Linear(in_features=8, out_features=1, bias=True)
  (logistic): Sigmoid()
)
Pretrained weights from checkpoints/t1_none_toytest.model are loaded!


In [35]:
############
## Target Market Evaluation data
############
tgt_task_generator = TaskGenerator(None, my_id_bank)
args.batch_size = 5000
tgt_valid_dataloader = tgt_task_generator.instance_a_market_valid_dataloader(args.tgt_market_valid, args.batch_size)
tgt_test_dataloader = tgt_task_generator.instance_a_market_valid_dataloader(args.tgt_market_test, args.batch_size)
print('loaded target test and validation data!')

loaded target test and validation data!


Here, we run the prediction step on both validation and test sets. 

Then, we write the output files in the format required for the submission and create the Zip file for submission.

Finally, we run the `validate_subsmission.py` to make sure that the structure of the Zip file is okay. In addition, we evaluate the model on the `valid` set.

**Note**: You need to run the script twice for both target markets (i.e., `t1` and `t2`). So, the code writes the prediction files in both `sample_run/t1/` and `sample_run/t2/` directories. Otherwise, your submission file will not pass the file structure test of `validate_submission.py`.

In [36]:
run_dir = './baseline_outputs/sample_run/'

def write_run_file(run_mf, file_address):
    with open(file_address, 'w') as fo:
        fo.write('userId\titemId\tscore\n')
        for u_id in run_mf:
            for p_id in run_mf[u_id]:
                fo.write('{}\t{}\t{}\n'.format(u_id, p_id, run_mf[u_id][p_id]))

valid_run_mf = mymodel.predict(tgt_valid_dataloader)
test_run_mf = mymodel.predict(tgt_test_dataloader)

In [37]:
write_run_file(valid_run_mf, path.join(run_dir, tgt_market, 'valid_pred.tsv'))
write_run_file(test_run_mf, path.join(run_dir, tgt_market, 'test_pred.tsv'))

# get full evaluation on validation set using pytrec_eval.
tgt_valid_qrel = read_qrel_file('DATA/{}/valid_qrel.tsv'.format(tgt_market))
task_ov, task_ind = get_evaluations_final(valid_run_mf, tgt_valid_qrel)

In [7]:
# Zip the run files into a single archive to prepare for submission    
! cd {run_dir} && zip -r ../sample_run.zip ./

print("*** Validating the submission Zip file ***")
# Run the validate_submission.py script to check if the file format is okay and get the performance on validation set.
! python validate_submission.py ./baseline_outputs/sample_run.zip

updating: t2/ (stored 0%)
updating: t2/test_pred.tsv (deflated 68%)
updating: t2/valid_pred.tsv (deflated 68%)
updating: t1/ (stored 0%)
updating: t1/valid_pred.tsv (deflated 67%)
updating: t1/test_pred.tsv (deflated 67%)
  adding: .DS_Store (deflated 95%)
  adding: t2/.DS_Store (deflated 97%)
  adding: t1/.DS_Store (deflated 97%)
*** Validating the submission Zip file ***
Extracting the submission zip file
Validating the file structure of the submission
File structure validation successfully passed
Evaluating the validation set
===================== Market : t1=====================
======= Set val : score(ndcg10_val)=0.319351407707 =======
======= Set val : score(r10_val)=0.497589914720 =======
===================== Market : t2=====================
======= Set val : score(ndcg10_val)=0.320296306874 =======
======= Set val : score(r10_val)=0.485041955491 =======
===================== Market : t1t2=====================
======= Set val : score(ndcg10_val)=0.319984729291 =======
======= S

# Baselines
Here we create the Zip file and validate the submission for all the baseline runs.

In [10]:
# no_src baseline
baseline_dir = './baseline_outputs/no_src_run/'
! cd {baseline_dir} && zip -r ../no_src.zip ./

print("*** Validating the submission Zip file ***")
# Run the validate_submission.py script to check if the file format is okay and get the performance on validation set.
! python validate_submission.py ./baseline_outputs/no_src.zip

  adding: .DS_Store (deflated 95%)
  adding: t2/ (stored 0%)
  adding: t2/.DS_Store (deflated 97%)
  adding: t2/test_pred.tsv (deflated 68%)
  adding: t2/valid_pred.tsv (deflated 68%)
  adding: t1/ (stored 0%)
  adding: t1/.DS_Store (deflated 97%)
  adding: t1/test_pred.tsv (deflated 67%)
  adding: t1/valid_pred.tsv (deflated 67%)
*** Validating the submission Zip file ***
Extracting the submission zip file
Validating the file structure of the submission
File structure validation successfully passed
Evaluating the validation set
===================== Market : t1=====================
======= Set val : score(ndcg10_val)=0.319351407707 =======
======= Set val : score(r10_val)=0.497589914720 =======
===================== Market : t2=====================
======= Set val : score(ndcg10_val)=0.320296306874 =======
======= Set val : score(r10_val)=0.485041955491 =======
===================== Market : t1t2=====================
======= Set val : score(ndcg10_val)=0.319984729291 =======
======= S

In [11]:
# s1 baseline
baseline_dir = './baseline_outputs/s1_run/'
! cd {baseline_dir} && zip -r ../s1.zip ./

print("*** Validating the submission Zip file ***")
# Run the validate_submission.py script to check if the file format is okay and get the performance on validation set.
! python validate_submission.py ./baseline_outputs/s1.zip

  adding: .DS_Store (deflated 96%)
  adding: t2/ (stored 0%)
  adding: t2/.DS_Store (deflated 97%)
  adding: t2/test_pred.tsv (deflated 68%)
  adding: t2/valid_pred.tsv (deflated 68%)
  adding: t1/ (stored 0%)
  adding: t1/.DS_Store (deflated 97%)
  adding: t1/test_pred.tsv (deflated 71%)
  adding: t1/valid_pred.tsv (deflated 71%)
*** Validating the submission Zip file ***
Extracting the submission zip file
Validating the file structure of the submission
File structure validation successfully passed
Evaluating the validation set
===================== Market : t1=====================
======= Set val : score(ndcg10_val)=0.291923073785 =======
======= Set val : score(r10_val)=0.492769744160 =======
===================== Market : t2=====================
======= Set val : score(ndcg10_val)=0.296293997024 =======
======= Set val : score(r10_val)=0.466982852973 =======
===================== Market : t1t2=====================
======= Set val : score(ndcg10_val)=0.294852698580 =======
======= S

In [12]:
# no_src baseline
baseline_dir = './baseline_outputs/s1_s2_s3_run/'
! cd {baseline_dir} && zip -r ../s1_s2_s3.zip ./

print("*** Validating the submission Zip file ***")
# Run the validate_submission.py script to check if the file format is okay and get the performance on validation set.
! python validate_submission.py ./baseline_outputs/s1_s2_s3.zip

  adding: .DS_Store (deflated 96%)
  adding: t2/ (stored 0%)
  adding: t2/.DS_Store (deflated 97%)
  adding: t2/test_pred.tsv (deflated 68%)
  adding: t2/valid_pred.tsv (deflated 68%)
  adding: t1/ (stored 0%)
  adding: t1/.DS_Store (deflated 97%)
  adding: t1/test_pred.tsv (deflated 69%)
  adding: t1/valid_pred.tsv (deflated 69%)
*** Validating the submission Zip file ***
Extracting the submission zip file
Validating the file structure of the submission
File structure validation successfully passed
Evaluating the validation set
===================== Market : t1=====================
======= Set val : score(ndcg10_val)=0.296685062055 =======
======= Set val : score(r10_val)=0.508342602892 =======
===================== Market : t2=====================
======= Set val : score(ndcg10_val)=0.125686247681 =======
======= Set val : score(r10_val)=0.147599288678 =======
===================== Market : t1t2=====================
======= Set val : score(ndcg10_val)=0.230883282893 =======
======= S